In [1]:
%load_ext autotime

In [2]:
import os

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook as tqdm

from piaa.utils import pipeline
from piaa.utils import helpers
from piaa.utils import noise

from pocs.utils.images import fits as fits_utils

from astropy.stats import sigma_clip, sigma_clipped_stats
from astropy import units as u

time: 1.83 s


#### Observation sources

We use the previously detected and filtered sources to examine our target. See the [Source-Detection](Source-Detection-And-Filtering.ipynb) for details.

In [3]:
base_dir = '/var/panoptes/images/fields/PAN012/358d0f/20180822T035809/'
# base_dir = '/var/panoptes/images/fields/PAN001/Hd189733/14d3bd/20180913T085704/'
source_filename = os.path.join(base_dir, f'point-sources-filtered.csv')

sources = pipeline.lookup_sources_for_observation(filename=source_filename).set_index(['picid'], append=True)

time: 886 ms


For our target table we want to compare the flux as calculated from `sextractor` (which has no knowledge of the RGB array) with that pulled from a custom stamp.  `sextractor` uses a 6-pixel circular aperture  centered around the calculated centroid (the `x` and `y` values in our `sources` table) while our custom stamps will use an aperture that is sliced according to our rule of having a fixed pixel pattern. Here we use a 6-pixel aperture on our stamps for direct comparison with `sextractor` but the size can be vary.

In [4]:
# Lookup exposure time used
exptime = fits_utils.getval(os.path.join(base_dir, sources.iloc[0].file), 'EXPTIME')

num_sources = len(sources.index.levels[1])

time: 9.77 ms


In [5]:
counts = list()

i = 0
for pid, target_table in tqdm(sources.groupby('picid'), total=num_sources): 
    i += 1
#     if i > 10:
#         break

    # Get PSC info for this target
    psc = pd.read_csv(os.path.join(base_dir, 'stamps', f'{pid}.csv'), parse_dates=True).set_index(['obs_time', 'picid'])
    
    # Loop through each frame of the PSC
    for idx, row in psc.iterrows():
        date_obs= idx[0]
        data = row.values.reshape(10, 10)
        
        row_info = dict(date_obs=date_obs, picid=pid, channel='all')
    
        try:
            stamp_noise = noise.get_stamp_noise(data, exptime, return_detail=True)
            
            stamp_noise.update({
                'date_obs': date_obs,
                'picid': pid,            
                'channel': 'all',            
            })
            counts.append(stamp_noise)
        except ValueError:
            continue
        
        # Calculate some properties for each channel
        color_data = helpers.get_rgb_data(data, force_new=True)
        for color, color_stamp in zip('rgb', color_data):
            
            try:
                stamp_noise = noise.get_stamp_noise(color_stamp, exptime, return_detail=True)
            except ValueError:
                continue
                
            stamp_noise.update({
                'date_obs': date_obs,
                'picid': pid,            
                'channel': color,            
            })
            counts.append(stamp_noise)            
                            
source_noise = pd.DataFrame(counts).set_index(['date_obs'])
source_noise.index = pd.to_datetime(source_noise.index)
source_noise.to_csv(os.path.join(base_dir, f'point-sources-noise.csv'))


time: 44min 45s
